In [1]:
import brightway2 as bw

In [2]:
bw.projects.set_current('Dimension reduction')

In [3]:
eidb = bw.Database("ecoinvent 3.6 cutoff")
random_act = eidb.random()
random_act

'market for jatropha seed' (kilogram, GLO, None)

In [4]:
lca = bw.LCA({random_act: 1}, method=('IPCC 2013', 'climate change', 'GWP 100a'))
lca.lci()
lca.lcia()

In [5]:
bio = bw.Database("biosphere3")

In [6]:
from collections import defaultdict
from pyprind import prog_bar
import numpy as np
from scipy import sparse

In [7]:
bio_grouping = defaultdict(list)
bio_grouping

defaultdict(list, {})

In [8]:
n = 0
for ds in bio:
    if ds not in lca.biosphere_dict:
        n = n+1 # skipped flows in biosphere
        # Not produced by any ecoinvent activity; skip
        continue

    bio_grouping[(ds['name'], ds['categories'][0])].append(ds)
    
len(bio), len(bio_grouping),n

(4321, 1317, 2244)

In [9]:
rows, cols, data = [], [], []
reduction_matrix_mapping = {}

for row_index, key in enumerate(sorted(bio_grouping)):
    for ds in bio_grouping[key]:
        rows.append(row_index)
        cols.append(lca.biosphere_dict[ds])
        data.append(1)
        reduction_matrix_mapping[key] = row_index
        
matrix = sparse.coo_matrix(
    (np.array(data), (np.array(rows), np.array(cols))),
    shape=(max(rows) + 1, max(cols) + 1)
).tocsr()

In [10]:
lca.biosphere_matrix.sum(), (matrix * lca.biosphere_matrix).sum()

(80272109619.25703, 80272109619.25713)

In [12]:
lca.characterized_inventory.sum(), (matrix * lca.characterized_inventory).sum()

(1.110456082285161, 1.1104560822851592)

In [13]:
vectors = {}

In [14]:
for ds in prog_bar(eidb):
    lca.redo_lcia({ds: 1})
    if not lca.score:
        continue
    vectors[lca.activity_dict[ds.key]] = np.ravel((matrix * lca.inventory.copy()).sum(axis=1))

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:44


In [15]:
len(vectors)

17313

In [16]:
import pandas as pd
df = pd.DataFrame(vectors)
df_new = df.T.copy()
df_new.save('activity_bioflows_inventory.csv')